In [1]:
import numpy as np
import pandas as pd
import string
import re
import random
import pickle
import os

with open(r'D:\smarterdevOCRScanner\1_WorkOrderNER\TSV FILES\workOrders_alandemel.txt', mode='r', encoding='utf8', errors='ignore') as f:
    text = f.read()

data = list(map(lambda x:x.split('\t'),text.split('\n')))

df = pd.DataFrame(data[1:],columns=data[0])

df.head(10)

,id,text,tag
0,Alandemel-01_page_1.png,/‘Nian,O
1,Alandemel-01_page_1.png,cde,O
2,Alandemel-01_page_1.png,Miaic,O
3,Alandemel-01_page_1.png,,O
4,Alandemel-01_page_1.png,WORKS,O
5,Alandemel-01_page_1.png,ORDER,O
6,Alandemel-01_page_1.png,To:,O
7,Alandemel-01_page_1.png,|,O
8,Alandemel-01_page_1.png,D&S,O
9,Alandemel-01_page_1.png,Appliances,O


In [2]:
### Cleaning Text - Remove white space - Remove Unwanted special characters
whitespace = string.whitespace
punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
tableWhitespace = str.maketrans('','',whitespace)
tablePunctuation = str.maketrans('','',punctuation)
def cleanText(txt):
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

df['text'] = df['text'].apply(cleanText)
dataClean = df.query("text != '' ")
dataClean.dropna(inplace=True)
dataClean.head(10)


C:\Users\USER\AppData\Local\Temp\ipykernel_29284\2534761583.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataClean.dropna(inplace=True)


,id,text,tag
0,Alandemel-01_page_1.png,/‘nian,O
1,Alandemel-01_page_1.png,cde,O
2,Alandemel-01_page_1.png,miaic,O
4,Alandemel-01_page_1.png,works,O
5,Alandemel-01_page_1.png,order,O
6,Alandemel-01_page_1.png,to,O
8,Alandemel-01_page_1.png,ds,O
9,Alandemel-01_page_1.png,appliances,O
11,Alandemel-01_page_1.png,date,B-ORDERDATE
12,Alandemel-01_page_1.png,2/11/2023,I-ORDERDATE


In [3]:
### Convert Data into Spacy Format

group = dataClean.groupby(by='id')
workorders = group.groups.keys()
allWorkOrdersData = []
for workorder in workorders:
    workOrderData = []
    grouparray = group.get_group(workorder)[['text','tag']].values
    content = ''
    annotations = {'entities':[]}
    start = 0
    end = 0
    for text, label in grouparray:
        text = str(text)
        stringLength = len(text) + 1

        start = end
        end = start + stringLength

        if label != 'O':
            annot = (start,end-1,label)
            annotations['entities'].append(annot)

        content = content + text + ' '
        
        
    workOrderData = (content,annotations)
    allWorkOrdersData.append(workOrderData)


allWorkOrdersData


[('/‘nian cde miaic works order to ds appliances date 2/11/2023 mr stedeford order bros-15337681 136-137 high street ilfracombe ordered by jeri davison bromley pmc ex34 9ez con.fax access tenant key return keys please execute the appliance - repairs at 656808 property 656808 flat 1 st. andrews house 72 anglesea road orpington br5 4aw contact tel contact mrs corina nechifor -7551823553 washing machine stops working mid cycle please advise what can be done. stoptap please contact the office below with a quotation prior to carrying out any works. required by as this works order has been placed on behalf of the "landlord," your invoice must be addressed directly to the landlord miss claire gina charleton c/o bromley adm and alan de maid bromley 19 east "street," bromley kent br1 1qe _ please email 7 send all invoices to the office address below office address 19 east "street," "bromley," "kent," br1 1qe phone 208 466 7172 fax 208 4666331 e-mail lettings@alandemaid.co.uk ',
  {'entities': [

In [4]:
## Split the Data into Training and Testing Set
# Assuming allWorkOrdersData is already loaded
random.shuffle(allWorkOrdersData)

# Split the data into training and testing sets
TrainData = allWorkOrdersData[:240]
TestData = allWorkOrdersData[240:]

# Ensure the 'data' directory exists
os.makedirs('./data', exist_ok=True)

# Save the training and testing data as pickle files
with open('./data/TrainData.pickle', mode='wb') as f_train:
    pickle.dump(TrainData, f_train)

with open('./data/TestData.pickle', mode='wb') as f_test:
    pickle.dump(TestData, f_test)